In [101]:
import pandas as pd
import numpy as np
import math


url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip"
df_music = pd.read_csv(url)
df_music.shape

(232725, 18)

In [102]:
df_music.loc[df_music['track_name'] == "7 rings"]

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
9027,Dance,Ariana Grande,7 rings,14msK75pk3pA33pzPVNtBF,100,0.578,0.725,178640,0.321,0.0,C#,0.0884,-10.744,Minor,0.323,70.142,4/4,0.319
107804,Pop,Ariana Grande,7 rings,14msK75pk3pA33pzPVNtBF,100,0.578,0.725,178640,0.321,0.0,C#,0.0884,-10.744,Minor,0.323,70.142,4/4,0.319


In [103]:
# There is two genre with similar name: "Children’s Music" and "Children's Music". I rename it before a merge with the following get dummies
df_music["genre"].replace('Children’s Music', "Children's Music", inplace=True)

# We remove Soundtrack and Movie genre
df_music_dumies = df_music[(df_music['genre'] != 'Soundtrack') & (df_music['genre'] != 'Movie')]

# Get dummies on genre for Machine Learning Usage
df_music_dumies = pd.concat([df_music_dumies , df_music_dumies['genre'].str.get_dummies()], axis = 1)

# Factorization of "mode"
df_music_dumies['mode'] = df_music_dumies['mode'].factorize()[0]

# We remove song with a duration below 90 seconds et above 900 seconds
df_music_dumies = df_music_dumies.loc[df_music_dumies['duration_ms'] >= 90000]
df_music_dumies = df_music_dumies.loc[df_music_dumies['duration_ms'] <= 900000]

In [104]:
# Function to split the popularity in 5 classes in order to use a classification Machine Learning method (suggested by Tarik as he says it's better than Linear Regression here)
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [105]:
# Applying the function
df_music_dumies["popularity_score"] = df_music_dumies["popularity"].apply(popularity_score)

In [106]:
# We remove useless columns
cols = ['genre', 'artist_name', 'track_id', 'key', 'time_signature', 'popularity']
df_music_dumies.drop(cols, axis=1, inplace=True)

In [107]:
# There are cells with a popularity of 0 and after many tests it seems we have better Machine Learning accuracy score when we remove it
df_music_without_0_pop = df_music_dumies[df_music_dumies['popularity_score'] != 0]

In [108]:
df_music_without_0_pop.to_csv('..\data\dataset_algo.csv.zip', header = True, compression='zip')

In [109]:
df_music_without_0_pop["popularity_score"].value_counts()

3    92238
2    61713
4    31368
1    19181
5      988
Name: popularity_score, dtype: int64